In [76]:
import os
import warnings
warnings.simplefilter(action="ignore", category=FutureWarning)

In [77]:
from seach_countries import mylist as search_contries
import numpy as np
import pandas as pd

In [78]:
def merge_datafraps(frames: list) -> pd.DataFrame:
    dataframe = frames[0]
    for frame in frames:
        processed_frame = frame.reset_index(drop=True)
        dataframe = pd.merge(dataframe, processed_frame)
        dataframe.sort_values(by="date").reset_index(drop=True, inplace=True)
    sorted_dataframe =  dataframe.sort_values(by="date").reset_index(drop=True)
    return sorted_dataframe

In [79]:
files = []

for file in os.listdir("trends"):
    if file.endswith(".csv"):
        filename = os.path.join("trends", file)
        files.append(pd.read_csv(filename))

In [80]:
bigboy = merge_datafraps(files)

In [81]:
bigboy.dropna(inplace=True)

In [82]:
bigboy.head()

,date,country,Ebola,Cholera,Zika virus,Measles,Malaria,Trauma,Racism,Loneliness,...,Climate Change,Global Warming,Pollution,Fossil Fuels,Renewable Energy,Facebook,YouTube,Instagram,Twitter,Snapchat
0,2010-01-01,Ecuador,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,44.0,75.0,0.0,0.0,0.0
1,2010-01-01,Estonia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,52.0,94.0,0.0,16.0,0.0
2,2010-01-01,Colombia,0.0,0.0,0.0,0.0,0.0,49.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,77.0,46.0,0.0,0.0,0.0
3,2010-01-01,Venezuela,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,84.0,24.0,0.0,2.0,0.0
4,2010-01-01,Argentina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,67.0,41.0,0.0,1.0,0.0


In [83]:
bigboy.tail()

,date,country,Ebola,Cholera,Zika virus,Measles,Malaria,Trauma,Racism,Loneliness,...,Climate Change,Global Warming,Pollution,Fossil Fuels,Renewable Energy,Facebook,YouTube,Instagram,Twitter,Snapchat
278279,2019-12-31,Peru,7.0,0.0,7.0,0.0,14.0,40.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,85.0,52.0,12.0,3.0,0.0
278280,2019-12-31,New Zealand,0.0,0.0,0.0,2.0,0.0,7.0,0.0,3.0,...,5.0,3.0,5.0,0.0,0.0,89.0,60.0,26.0,7.0,4.0
278281,2019-12-31,Netherlands,0.0,0.0,1.0,0.0,2.0,24.0,0.0,8.0,...,15.0,10.0,10.0,0.0,5.0,89.0,79.0,34.0,15.0,3.0
278282,2019-12-31,Colombia,2.0,0.0,2.0,0.0,2.0,5.0,0.0,5.0,...,74.0,0.0,25.0,0.0,49.0,83.0,72.0,22.0,7.0,1.0
278283,2019-12-31,Venezuela,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,72.0,53.0,13.0,6.0,0.0


In [84]:
bigboy.to_csv("bigboy.csv", index=False)

In [85]:
terms = bigboy.drop(columns=["date", "country"]).columns.tolist()

padded_row = pd.DataFrame(np.zeros((1, len(terms))), columns=terms)

In [86]:
padded_row

,Ebola,Cholera,Zika virus,Measles,Malaria,Trauma,Racism,Loneliness,Violence,Anxiety,...,Climate Change,Global Warming,Pollution,Fossil Fuels,Renewable Energy,Facebook,YouTube,Instagram,Twitter,Snapchat
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [87]:
norDB = bigboy[bigboy["country"] == "Norway"]

In [88]:
valid_dated = norDB["date"].unique()

In [89]:
valid_dated.shape

(3652,)

In [100]:
for cont in search_contries:
    tmp = bigboy[bigboy["country"] == cont]
    if tmp.shape != (3652, 52):

        # find missing dates
        missing_dates = np.setdiff1d(valid_dated, tmp["date"].unique())

        pri = missing_dates.shape[0]
        
        # insert zeros at missing dates
        for date in missing_dates:
            vals = {"country": [cont], "date": [date]}
            for term in terms:
                vals[term] = 0.0
            add_frame = pd.DataFrame(data=vals)
            # tmp = pd.concat([tmp, add_frame])
            tmp["date"] = pd.to_datetime(tmp["date"])
            add_frame["date"] = pd.to_datetime(add_frame["date"])
            tmp = pd.concat([tmp.reset_index(drop=True), add_frame.reset_index(drop=True)])
            tmp.sort_values(by="date").reset_index(drop=True, inplace=True)

        post = np.setdiff1d(valid_dated, tmp["date"].unique())

        print(f"{cont} had {pri} missing dates, now has {post.shape[0]}")
        tmp.reset_index(drop=True, inplace=True)
        bigboy[bigboy["country"] == cont] = tmp
        


/tmp/ipykernel_209479/1810191326.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["date"] = pd.to_datetime(tmp["date"])


Afghanistan had 730 missing dates, now has 3652


/tmp/ipykernel_209479/1810191326.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["date"] = pd.to_datetime(tmp["date"])


Albania had 365 missing dates, now has 3652


/tmp/ipykernel_209479/1810191326.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["date"] = pd.to_datetime(tmp["date"])


...

Georgia had 365 missing dates, now has 3652


/tmp/ipykernel_209479/1810191326.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["date"] = pd.to_datetime(tmp["date"])


Libya had 730 missing dates, now has 3652
Liechtenstein had 3652 missing dates, now has 3652


/tmp/ipykernel_209479/1810191326.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  tmp["date"] = pd.to_datetime(tmp["date"])


Mongolia had 365 missing dates, now has 3652
Montenegro had 3652 missing dates, now has 3652


In [101]:
for cont in search_contries:
    tmp = bigboy[bigboy["country"] == cont]
    print(cont, tmp.shape)

Afghanistan (0, 52)
Albania (0, 52)
Argentina (3652, 52)
Australia (3652, 52)
Austria (3652, 52)
Bangladesh (3652, 52)
Belarus (0, 52)
Belgium (3652, 52)
Bolivia (3652, 52)
Bosnia & Herzegovina (3652, 52)
Brazil (3652, 52)
Cameroon (3652, 52)
Canada (3652, 52)
Chile (3652, 52)
China (3652, 52)
Colombia (3652, 52)
Croatia (3652, 52)
Cuba (3652, 52)
Czechia (3652, 52)
Denmark (3652, 52)
Ecuador (3652, 52)
Egypt (3652, 52)
Estonia (3652, 52)
Finland (3652, 52)
France (3652, 52)
Gambia (0, 52)
Georgia (0, 52)
Germany (3652, 52)
Greece (3652, 52)
Hong Kong (3652, 52)
Hungary (3652, 52)
Iceland (3652, 52)
India (3652, 52)
Iran (3652, 52)
Iraq (3652, 52)
Ireland (3652, 52)
Israel (3652, 52)
Italy (3652, 52)
Japan (3652, 52)
Kenya (3652, 52)
Latvia (3652, 52)
Lebanon (3652, 52)
Libya (0, 52)
Liechtenstein (0, 52)
Lithuania (3652, 52)
Luxembourg (3652, 52)
Mexico (3652, 52)
Mongolia (0, 52)
Montenegro (0, 52)
Morocco (3652, 52)
Netherlands (3652, 52)
New Zealand (3652, 52)
Nigeria (3652, 52)
No

In [67]:
bigboy.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 278284 entries, 0 to 278283
Data columns (total 52 columns):
 #   Column                   Non-Null Count   Dtype  
---  ------                   --------------   -----  
 0   date                     259292 non-null  object 
 1   country                  259292 non-null  object 
 2   Ebola                    259292 non-null  float64
 3   Cholera                  259292 non-null  float64
 4   Zika virus               259292 non-null  float64
 5   Measles                  259292 non-null  float64
 6   Malaria                  259292 non-null  float64
 7   Trauma                   259292 non-null  float64
 8   Racism                   259292 non-null  float64
 9   Loneliness               259292 non-null  float64
 10  Violence                 259292 non-null  float64
 11  Anxiety                  259292 non-null  float64
 12  Big Data                 259292 non-null  float64
 13  Artificial Intelligence  259292 non-null  float64
 14  Augm

In [68]:
bigboy.head()

,date,country,Ebola,Cholera,Zika virus,Measles,Malaria,Trauma,Racism,Loneliness,...,Climate Change,Global Warming,Pollution,Fossil Fuels,Renewable Energy,Facebook,YouTube,Instagram,Twitter,Snapchat
0,2010-01-01,Ecuador,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,44.0,75.0,0.0,0.0,0.0
1,2010-01-01,Estonia,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,52.0,94.0,0.0,16.0,0.0
2,2010-01-01,Colombia,0.0,0.0,0.0,0.0,0.0,49.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,77.0,46.0,0.0,0.0,0.0
3,2010-01-01,Venezuela,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,84.0,24.0,0.0,2.0,0.0
4,2010-01-01,Argentina,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,67.0,41.0,0.0,1.0,0.0


In [69]:
bigboy.tail()

,date,country,Ebola,Cholera,Zika virus,Measles,Malaria,Trauma,Racism,Loneliness,...,Climate Change,Global Warming,Pollution,Fossil Fuels,Renewable Energy,Facebook,YouTube,Instagram,Twitter,Snapchat
278279,2019-12-31,Peru,7.0,0.0,7.0,0.0,14.0,40.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,85.0,52.0,12.0,3.0,0.0
278280,2019-12-31,New Zealand,0.0,0.0,0.0,2.0,0.0,7.0,0.0,3.0,...,5.0,3.0,5.0,0.0,0.0,89.0,60.0,26.0,7.0,4.0
278281,2019-12-31,Netherlands,0.0,0.0,1.0,0.0,2.0,24.0,0.0,8.0,...,15.0,10.0,10.0,0.0,5.0,89.0,79.0,34.0,15.0,3.0
278282,2019-12-31,Colombia,2.0,0.0,2.0,0.0,2.0,5.0,0.0,5.0,...,74.0,0.0,25.0,0.0,49.0,83.0,72.0,22.0,7.0,1.0
278283,2019-12-31,Venezuela,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,72.0,53.0,13.0,6.0,0.0


In [72]:
bigboy.isna().sum()

date                       18992
country                    18992
Ebola                      18992
Cholera                    18992
Zika virus                 18992
Measles                    18992
Malaria                    18992
Trauma                     18992
Racism                     18992
Loneliness                 18992
Violence                   18992
Anxiety                    18992
Big Data                   18992
Artificial Intelligence    18992
Augmented Reality          18992
Internet of Things         18992
Cloud Computing            18992
Unemployment               18992
Criminality                18992
Corruption                 18992
Poverty                    18992
Prison                     18992
Osama Bin Laden            18992
Donald Trump               18992
Brexit                     18992
ISIS                       18992
al-Qaida                   18992
Elon Musk                  18992
Jeff Bezos                 18992
Bill Gates                 18992
Warren Buf

In [73]:
bigboy.to_csv("cleanbigboy.csv", index=False)
